# Ejercicios Tema 9

---

`Ejercicio 2 para casa`

>*La agregación limitada por difusión (DLA) es un proceso de crecimiento de grupos de partículas agregadas impulsado por su difusión aleatoria. Hay dos tipos de partículas, como en el modelo de segregación de Schelling, pero solo una de ellas puede moverse libremente. Las partículas móviles se difunden en un espacio bidimensional al azar, mientras que las partículas inmóviles no hacen nada; simplemente permanecen donde están. Si una partícula móvil choca con una partícula inamovible (es decir, si se acercan lo suficiente entre sí), la partícula móvil se vuelve inamovible y permanece allí para siempre. Esta es la única regla para la interacción de los agentes.
Implementa el código de simulador del modelo DLA y realiza una simulación con todas las partículas inicialmente móviles, excepto una partícula "semilla" inamovible colocada en el centro del espacio, y observar el tipo de patrón espacial que emerge. Realiza simulaciones con múltiples semillas inamovibles colocadas aleatoriamente en el espacio, y observe cómo interactúan entre sí a escalas macroscópicas.*

In [2]:
import matplotlib
matplotlib.use('TkAgg')
from pylab import *
import simulador

# Parámetros del modelo
n = 30              # número de partículas móviles
r = 0.05            # radio de difusión
inm_ini = 1         # número de particulas inamovibles
inm_radio = 0.1     # radio de las particulas inamovibles


class Particle:
    pass


def initialize():
    global particles
    particles = []
    
    # Particulas inamovibles
    for i in range(inm_ini):
        seed = Particle()
        seed.x = random()
        seed.y = random()
        seed.es_inmovible = True
        particles.append(seed)
    
    #  Particulas móviles
    for i in range(n):
        particle = Particle()
        particle.x = random()
        particle.y = random()
        particle.es_inmovible = False
        particles.append(particle)

def observe():
    global particles
    cla()
    inmovible = [p for p in particles if p.es_inmovible]
    moviles = [p for p in particles if not p.es_inmovible]
    plot([p.x for p in inmovible], [p.y for p in inmovible], 'ro')
    plot([p.x for p in moviles], [p.y for p in moviles], 'bo')
    axis('image')
    axis([0, 1, 0, 1])

def update():
    global particles
    for particle in particles:
        if not particle.es_inmovible: # Si la partícula es movible

            particle.x += (random() - 0.5) * r
            particle.y += (random() - 0.5) * r

            # Verificar si choca
            for inmovibles in particles:
                if inmovibles.es_inmovible:
                    if (particle.x - inmovibles.x)**2 + (particle.y - inmovibles.y)**2 < inm_radio**2:
                        particle.es_inmovible = True
                        break

simulador.GUI().start(func=[initialize, observe, update])

El comportamiento de las particulas a escala macroscopica es el de concentrarse al rededor de la particula inamovible, creando una especie de nucleo circular.

---

`Ejercicio 5 para casa`

>*Este es otro modelo de ABM con una interacción agente-entorno, presentado por primera vez por Mitchel Resnick. Se supone la existencia de muchas piezas diminutas de basura esparcidas en un espacio 2D, donde existen hormigas que  deambulan al azar. Cuando una hormiga llega a un lugar donde hay basura, se comporta de acuerdo con las siguientes reglas:*
>1. Si la hormiga sostiene un pedazo de basura, lo deja caer.
>2. Si la hormiga no tiene basura, recoge un pedazo de basura.

>Implementa un ABM de este modelo y realizar simulaciones para averiguar qué tipo de comportamiento colectivo emerge. Si se implementa el modelo correctamente, se ve que estas reglas de comportamiento permiten que las hormigas recolecten y apilen basura de manera espontánea y limpien el espacio a largo plazo.*

In [2]:
import matplotlib
matplotlib.use('TkAgg')
from pylab import *
import simulador

# Parámetros del modelo
n = 20  # número de hormigas
m = 30  # número inicial de pedazos de basura
w = 100  # tamaño del espacio 2D

class Hormiga:
    pass

class Basura:
    pass

def initialize():
    global hormigas, basuras
    hormigas = []
    basuras = []
    
    for _ in range(n):
        hormiga = Hormiga()
        hormiga.x = randint(w)
        hormiga.y = randint(w)
        hormiga.tiene_basura = False
        hormigas.append(hormiga)
    
    for _ in range(m):
        basura = Basura()
        basura.x = randint(w)
        basura.y = randint(w)
        basuras.append(basura)


def observe():
    global hormigas, basuras
    cla()

    x_hormigas = [hormiga.x for hormiga in hormigas]
    y_hormigas = [hormiga.y for hormiga in hormigas]
    plot(x_hormigas, y_hormigas, 'r.')

    x_basuras = [basura.x for basura in basuras]
    y_basuras = [basura.y for basura in basuras]
    plot(x_basuras, y_basuras, 'b.')
    axis('image')
    axis([0, w, 0, w])


def update():
    global hormigas, basuras
    
    # Movimiento aleatorio de las hormigas
    for horm in hormigas:
        horm.x = (horm.x + randint(-1, 2)) % w
        horm.y = (horm.y + randint(-1, 2)) % w
    
    # Interacción de las hormigas con la basura
    for horm in hormigas:
        for bas in basuras:
            if horm.x == bas.x and horm.y == bas.y:
                # Si la hormiga ya tiene una basura
                if horm.tiene_basura:
                    # Dejar caer la basura anterior (crear una nueva)
                    nueva_bas = Basura()
                    nueva_bas.x = bas.x + randint(2) # En este caso se va soltando la basura hacia arriba a la derecha.
                    nueva_bas.y = bas.y + randint(2)
                    basuras.append(nueva_bas)

                    # Recoger la basura
                    basuras.remove(bas) 
                    horm.tiene_basura = True

                # Si la hormiga no tiene una basura
                else:
                    basuras.remove(bas)  # Recoger la basura
                    horm.tiene_basura = True


simulador.GUI().start(func=[initialize, observe, update])


En este caso, si hay mas hormigas que trozos de basura, cada hormiga recogerá un pedazo y se limpiará el espacio. De lo contrario, si hay mas basura que hormigas la basura se irá apilando en la parte superior derecha del espacio bidimensional (del modo en el que he realizado yo el modo de dejar caer la basura).